## ETL process

In this notebook, we go through the etl process, yada yada yada


Creating the database based on the ERD tables

## Creating our Databse Tables from Our ERD

In [32]:
import pandas as pd
import numpy as np
import sqlite3
import numpy as np
conn = sqlite3.connect('CourseData.db')
c = conn.cursor()

In [33]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [34]:
%%sql

sqlite:///CourseData.db

'Connected: @CourseData.db'

creating the separate tables.

Creating Programs Table

In [35]:
%%sql
DROP TABLE IF EXISTS PROGRAMS;

CREATE TABLE PROGRAMS ( 
    PID INTEGER PRIMARY KEY,
    PROGRAMCODE TEXT NOT NULL,
    PROGRAMNAME TEXT NOT NULL
);


 * sqlite:///CourseData.db
(sqlite3.OperationalError) attempt to write a readonly database [SQL: 'DROP TABLE IF EXISTS PROGRAMS;'] (Background on this error at: http://sqlalche.me/e/e3q8)


Creating Faculty Table

In [36]:
%%sql
DROP TABLE IF EXISTS FACULTY;
CREATE TABLE FACULTY ( 
    FID INTEGER PRIMARY KEY,
    LNAME TEXT NOT NULL,
    FNAME TEXT NOT NULL
);


 * sqlite:///CourseData.db
(sqlite3.OperationalError) attempt to write a readonly database [SQL: 'DROP TABLE IF EXISTS FACULTY;'] (Background on this error at: http://sqlalche.me/e/e3q8)


Creating the Location Table

In [37]:
%%sql
DROP TABLE IF EXISTS LOCATION;

CREATE TABLE LOCATION ( 
    LID INTEGER PRIMARY KEY,
    BUILDING TEXT,
    ROOM TEXT,
    LOCATION TEXT
);


 * sqlite:///CourseData.db
(sqlite3.OperationalError) attempt to write a readonly database [SQL: 'DROP TABLE IF EXISTS LOCATION;'] (Background on this error at: http://sqlalche.me/e/e3q8)


Creating the Course Table

In [38]:
%%sql
DROP TABLE IF EXISTS COURSE;
CREATE TABLE COURSE ( 
    CID INTEGER PRIMARY KEY,
    PID INTEGER NOT NULL,
    EFFECTIVETERM TEXT NOT NULL,
    CATALOG_ID TEXT NOT NULL,
    COURSE_TITLE TEXT NOT NULL,
    CREDITS TEXT NOT NULL,
    PREREQS TEXT,
    COREQS TEXT,
    FEES REAL,
    ATTRIBUTES TEXT,
    DESCRIPTION TEXT,
    FOREIGN KEY (PID) REFERENCES PROGRAMS (PID)
);

 * sqlite:///CourseData.db
(sqlite3.OperationalError) attempt to write a readonly database [SQL: 'DROP TABLE IF EXISTS COURSE;'] (Background on this error at: http://sqlalche.me/e/e3q8)


Creating teh Course_Offering Table

In [39]:
%%sql
DROP TABLE IF EXISTS COURSE_OFFERING;
CREATE TABLE COURSE_OFFERING ( 
    COID INTEGER PRIMARY KEY,
    CID INTEGER,
    FID INTEGER NOT NULL,
    CRN TEXT NOT NULL,
    SEMESTER TEXT NOT NULL,
    YEAR INTEGER NOT NULL,
    TITLE TEXT NOT NULL,
    CATALOG_ID TEXT,
    TERM TEXT,
    EffectiveTerm TEXT,
    MEETINGS TEXT,
    TIMECODES TEXT,
    SECTION TEXT NOT NULL,
    CAP INTEGER NOT NULL,
    ACT INTEGER NOT NULL,
    REM INTEGER NOT NULL,
    LNAME TEXT,
    FNAME TEXT,
    FOREIGN KEY (CID) REFERENCES COURSE (CID)
    FOREIGN KEY (FID) REFERENCES FACULTY (FID)
);


 * sqlite:///CourseData.db
(sqlite3.OperationalError) attempt to write a readonly database [SQL: 'DROP TABLE IF EXISTS COURSE_OFFERING;'] (Background on this error at: http://sqlalche.me/e/e3q8)


Creating the Class_Meetings Table

In [40]:
%%sql
DROP TABLE IF EXISTS CLASS_MEETINGS;
CREATE TABLE CLASS_MEETINGS ( 
    CMID INTEGER PRIMARY KEY,
    COID INTEGER,
    LID INTEGER,
    DAY TEXT,
    START TEXT,
    END TEXT,
    CRN TEXT,
    TERM TEXT,
    FOREIGN KEY (LID) REFERENCES LOCATION (LID)
    FOREIGN KEY (COID) REFERENCES COURSE_OFFERING (COID)
);


 * sqlite:///CourseData.db
(sqlite3.OperationalError) attempt to write a readonly database [SQL: 'DROP TABLE IF EXISTS CLASS_MEETINGS;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [41]:
%%sql 

DROP TABLE IF EXISTS CATALOG_YEAR;
CREATE TABLE CATALOG_YEAR (
    EFFECTIVETERM TEXT,
    YEAR TEXT,
    SEMESTER TEXT
);

INSERT INTO CATALOG_YEAR (Semester,Year,EffectiveTerm) VALUES 
('Fall','2014','2014-2015'),('Winter','2015','2014-2015'),('Spring','2015','2014-2015'),('Summer','2015','2014-2015'),
('Fall','2015','2015-2016'),('Winter','2016','2015-2016'),('Spring','2016','2015-2016'),('Summer','2016','2015-2016'),
('Fall','2016','2016-2017'),('Winter','2017','2016-2017'),('Spring','2017','2016-2017'),('SpringBreak','2017','2016-2017'),('Summer','2017','2016-2017'),
('Fall','2017','2017-2018'),('Winter','2018','2017-2018'),('Spring','2018','2017-2018'),('Summer','2018','2017-2018'),
('Fall','2018','2018-2019'),('Winter','2019','2018-2019'),('Spring','2019','2018-2019');

 * sqlite:///CourseData.db
(sqlite3.OperationalError) attempt to write a readonly database [SQL: 'DROP TABLE IF EXISTS CATALOG_YEAR;'] (Background on this error at: http://sqlalche.me/e/e3q8)


## Make and Populate our IMPORT tables from our CSV Files

In [42]:
%%sql 

SELECT * FROM CATALOG_YEAR;


 * sqlite:///CourseData.db
Done.


EFFECTIVETERM,YEAR,SEMESTER
2014-2015,2014,Fall
2014-2015,2015,Winter
2014-2015,2015,Spring
2014-2015,2015,Summer
2015-2016,2015,Fall
2015-2016,2016,Winter
2015-2016,2016,Spring
2015-2016,2016,Summer
2016-2017,2016,Fall
2016-2017,2017,Winter


In [43]:
%%sql

DROP TABLE IF EXISTS IMPORT_Courses;
DROP TABLE IF EXISTS IMPORT_Course_Meetings;
DROP TABLE IF EXISTS IMPORT_CourseCatlog;
DROP TABLE IF EXISTS IMPORT_Course_Catalog;

CREATE TABLE IMPORT_Courses (
    term TEXT,
    crn TEXT,
    catalog_id TEXT,
    section TEXT,
    credits TEXT,
    title TEXT,
    meetings TEXT,
    timecodes TEXT,
    primary_instructor TEXT,
    cap INT,
    act INT, 
    rem INT,
    fname TEXT,
    lname TEXT,
    semester TEXT,
    year TEXT
);


CREATE TABLE IMPORT_Course_Meetings(
    crn TEXT,
    location TEXT,
    day TEXT,
    start TEXT,
    end TEXT,
    term TEXT, 
    building TEXT, 
    room TEXT
);

CREATE TABLE IMPORT_Course_Catalog(
    program_code TEXT,
    program_name TEXT,
    catalog_ID TEXT,
    course_title TEXT,
    credits TEXT,
    prereqs TEXT,
    coreqs TEXT,
    fees TEXT,
    attributes TEXT,
    description TEXT,
    EffectiveTerm TEXT
);

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
(sqlite3.OperationalError) attempt to write a readonly database [SQL: 'CREATE TABLE IMPORT_Courses (\n    term TEXT,\n    crn TEXT,\n    catalog_id TEXT,\n    section TEXT,\n    credits TEXT,\n    title TEXT,\n    meetings TEXT,\n    timecodes TEXT,\n    primary_instructor TEXT,\n    cap INT,\n    act INT, \n    rem INT,\n    fname TEXT,\n    lname TEXT,\n    semester TEXT,\n    year TEXT\n);'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [ ]:
folder_names = ['Fall2014', 'Fall2015', 'Fall2016', 'Fall2017', 'Fall2018', 'Spring2015', 'Spring2016', 'Spring2017', 'Spring2018',
               'Spring2019', 'SpringBreak2017', 'Summer2015', 'Summer2016', 'Summer2017', 'Summer2018', 'Winter2015', 'Winter2016', 
               'Winter2017', 'Winter2018']


for file in folder_names:
    data = 'SourceData/' + file + '/course_meetings.csv'
    
    if (file == folder_names[0]):
        df = pd.read_csv(data)
    else:
        df = df.append(pd.read_csv(data), ignore_index=True)
    
    
    location = list(df.location)
    building = []
    room = []

    for loc in location:
        loc = loc.strip("\n ' '")
        data = loc.split(' ')

        length = len(data)

        if length == 2:
            building.append(data[0])
            room.append(data[1])
        else:
            building.append(data[0])
            room.append('NULL')


    df['building'] = building

    df['room'] = room
    
df.to_sql('IMPORT_Course_Meetings', conn, if_exists='append', index=False)

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
%%sql

SELECT COUNT() FROM IMPORT_Course_Meetings;

In [ ]:
folder_names = ['Fall2014', 'Fall2015', 'Fall2016', 'Fall2017', 'Fall2018', 'Spring2015', 'Spring2016', 'Spring2017', 'Spring2018',
               'Spring2019', 'SpringBreak2017', 'Summer2015', 'Summer2016', 'Summer2017', 'Summer2018', 'Winter2015', 'Winter2016', 
               'Winter2017', 'Winter2018']


for file in folder_names:
    data = 'SourceData/' + file + '/courses.csv'
    
    if (file == folder_names[0]):
        df = pd.read_csv(data)
    else:
        df = df.append(pd.read_csv(data), ignore_index=True)
        
        
        
    
    instructor = list(df.primary_instructor)
    lname = []
    fname = []

    for name in instructor:
        data = name.split('  ')

        length = len(data)

        if length == 2:
            fname.append(data[0])
            lname.append(data[1])
        else:
            fname.append('TBA')
            lname.append('TBA')


    df['fname'] = fname

    df['lname'] = lname
    
    
    term = list(df.term)
    semester = []
    year = []
    

    for t in term:
        data = t.split('2')
        semester.append(data[0])
        year_val = '2'+data[1]
        year.append(year_val)
        
    df['semester'] = semester
    df['year'] = year

df.to_sql('IMPORT_Courses', conn, if_exists='append', index=False)

In [ ]:
%%sql
SELECT * FROM IMPORT_COURSES LIMIT 10;

In [ ]:
folder_names = ['SourceData/Catalogs/CourseCatalog2017_2018.csv', 'SourceData/Catalogs/CourseCatalog2018_2019.csv']

df1 = pd.read_csv(folder_names[0])

df1['EffectiveTerm'] = 'NaN'

df1['EffectiveTerm'] = df1['EffectiveTerm'].replace('NaN', '2017-2018')

df2 = pd.read_csv(folder_names[1])

df2['EffectiveTerm'] = 'NaN'

df2['EffectiveTerm'] = df2['EffectiveTerm'].replace('NaN', '2018-2019')

df3 = df1.append(df2, ignore_index=True)


    
df3.to_sql('IMPORT_Course_Catalog', conn, if_exists='append', index=False)

## Populate the ERD Tables from Our Imported Data

In [ ]:
%%sql
INSERT INTO PROGRAMS (ProgramCode,ProgramName) 
SELECT DISTINCT program_code,program_name
FROM IMPORT_COURSE_CATALOG;

In [ ]:
%%sql

SELECT * FROM PROGRAMS
LIMIT 10;

In [ ]:
%%sql
INSERT INTO FACULTY (LName,FName)
SELECT DISTINCT LName, FName
FROM IMPORT_COURSES;

In [ ]:
%%sql
INSERT INTO LOCATION (Building,Room,Location) 
SELECT DISTINCT building,room,location
FROM IMPORT_COURSE_MEETINGS;

In [ ]:
%%sql
INSERT INTO COURSE (PID,EffectiveTerm,Catalog_ID,Course_Title,Credits,Prereqs,Coreqs,Fees,Attributes,Description)
SELECT DISTINCT PID,EffectiveTerm,Catalog_ID,Course_Title,Credits,Prereqs,Coreqs,Fees,Attributes,Description
FROM IMPORT_COURSE_CATALOG
    JOIN PROGRAMS ON (PROGRAMS.PROGRAMCODE=IMPORT_COURSE_CATALOG.program_code);

In [ ]:
%%sql

SELECT PID, Catalog_ID FROM COURSE
LIMIT 10;

In [ ]:
%%sql 

SELECT COUNT(credits) FROM IMPORT_Courses;

In [ ]:
%%sql
INSERT INTO COURSE_OFFERING (CID,FID,CRN,Semester,Year,Title,Catalog_ID,Term,EffectiveTerm,Meetings,Timecodes,Section,Cap,Act,Rem,lname,fname)
SELECT DISTINCT CID,FID,CRN,Semester,Year,Title,Catalog_ID,Term,EffectiveTerm,Meetings,Timecodes,Section,Cap,Act,Rem,lname,fname
FROM IMPORT_COURSES
    LEFT JOIN CATALOG_YEAR USING (Semester, Year)
    LEFT JOIN COURSE USING (EffectiveTerm, Catalog_ID)
    JOIN FACULTY USING (lname, fname);

In [ ]:
%%sql 

SELECT * FROM COURSE_OFFERING LIMIT 10;

In [ ]:
%%sql

SELECT COUNT(*) FROM LOCATION;


In [ ]:
%%sql 
INSERT INTO CLASS_MEETINGS (COID,LID,CRN,Day,Start,End,Term)
SELECT DISTINCT COURSE_OFFERING.COID,LOCATION.LID,COURSE_OFFERING.CRN,Day,Start,End,COURSE_OFFERING.Term
FROM IMPORT_COURSE_MEETINGS
    JOIN COURSE_OFFERING USING (CRN)
    JOIN LOCATION USING (Building,Room);

In [ ]:
%%sql
DROP TABLE IMPORT_COURSE_MEETINGS;
DROP TABLE IMPORT_COURSES;
DROP TABLE IMPORT_COURSE_CATALOG;

In [ ]:
c.close()
del c
conn.close()